In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

# Preparação dos dados

In [ ]:
(x_treinamento, y_treinamento), (_, _) = tf.keras.datasets.mnist.load_data()

In [ ]:
x_treinamento.shape

In [ ]:
x_treinamento.shape[1] * x_treinamento.shape[2]

In [ ]:
i = np.random.randint(0, x_treinamento.shape[0])
plt.imshow(x_treinamento[i], cmap = 'gray')
plt.title('É o numero ' + str(y_treinamento[i]))

In [ ]:
x_treinamento = x_treinamento.reshape((x_treinamento.shape[0], x_treinamento.shape[1], x_treinamento.shape[2], 1)).astype('float32')
x_treinamento.shape

In [ ]:
x_treinamento[0].min(), x_treinamento[0].max()

In [ ]:
meio_escala = x_treinamento[0].max() / 2
x_treinamento = (x_treinamento - meio_escala) / meio_escala

In [ ]:
buffer_size = x_treinamento.shape[0]
batch_size = 256

In [ ]:
buffer_size / batch_size

In [ ]:
type(x_treinamento)

In [ ]:
x_treinamento = tf.data.Dataset.from_tensor_slices(x_treinamento).shuffle(buffer_size).batch(batch_size)

In [ ]:
type(x_treinamento)

In [ ]:
x_treinamento

# Gerador

In [ ]:
# largura x altura x canais
neuronios_ocultos = 7 * 7 * 256

In [ ]:
def cria_gerador():
    network = tf.keras.Sequential()
    network.add(layers.Dense(units = neuronios_ocultos, use_bias = False, input_shape = (100, )))
    network.add(layers.BatchNormalization())
    network.add(layers.LeakyReLU())

    network.add(layers.Reshape((7, 7, 256)))

    # 7x7x128
    network.add(layers.Conv2DTranspose(filters = 128, kernel_size = 5, padding = 'same', use_bias = False))
    network.add(layers.BatchNormalization())
    network.add(layers.LeakyReLU())

    # 14x14x64
    network.add(layers.Conv2DTranspose(filters = 64, kernel_size = 5, strides = 2, padding = 'same', use_bias = False))
    network.add(layers.BatchNormalization())
    network.add(layers.LeakyReLU())

    # 28x28x1
    network.add(layers.Conv2DTranspose(filters = 1, kernel_size = 5, strides = 2, padding = 'same', use_bias = False, activation = 'tanh'))
    network.summary()

    return network

In [ ]:
gerador = cria_gerador()
gerador.input

In [ ]:
ruido = tf.random.normal([1, 100])
ruido

In [ ]:
imagem_gerada = gerador(ruido, training = False)
imagem_gerada.shape

In [ ]:
plt.imshow(imagem_gerada[0, :, :, 0], cmap = 'gray')
plt.title('Imagem gerada pelo gerador')

# Discriminador

In [ ]:
def cria_discriminador():
    network = tf.keras.Sequential()

    # 14x14x64
    network.add(layers.Conv2D(filters = 64, kernel_size = 5, padding = 'same', input_shape = [28, 28, 1], strides = 2))
    network.add(layers.LeakyReLU())
    network.add(layers.Dropout(0.3))

    # 7x7x128
    network.add(layers.Conv2D(filters = 128, kernel_size = 5, padding = 'same', strides = 2))
    network.add(layers.LeakyReLU())
    network.add(layers.Dropout(0.3))

    network.add(layers.Flatten())
    network.add(layers.Dense(units = 1))

    network.summary()
    return network

In [ ]:
discriminador = cria_discriminador()
discriminador.input


In [ ]:
discriminador(imagem_gerada, training = False) # logits

In [ ]:
tf.sigmoid(discriminador(imagem_gerada, training = False)).numpy() # probabilidade

# Calculo do erro

In [ ]:
tf.ones_like(1)

In [ ]:
tf.ones_like(0)

In [ ]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits = True)

In [ ]:
def descriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)

    total_loss = real_loss + fake_loss

    return total_loss

In [ ]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)


In [ ]:
gerador_otimizador = tf.keras.optimizers.Adam(1e-4)
discriminador_otimizador = tf.keras.optimizers.Adam(1e-4)

# Treinamento

In [ ]:
x_treinamento

In [ ]:
epochs = 100
ruido_dimensao = 100
numero_exemplos_para_gerar = 16

In [ ]:
batch_size, ruido_dimensao

In [ ]:
@tf.function
def treinamento(imagens):
  ruido = tf.random.normal([batch_size, ruido_dimensao])

  with tf.GradientTape() as gerador_tape, tf.GradientTape() as discriminador_tape:
    imagens_geradas = gerador(ruido, training = True)

    expected_output = discriminador(imagens, training = True)
    fake_output = discriminador(imagens_geradas, training = True)

    gerador_loss = generator_loss(fake_output)
    discriminador_loss = descriminator_loss(expected_output, fake_output)

  gradientes_gerador = gerador_tape.gradient(gerador_loss, gerador.trainable_variables)
  gradientes_discriminador = discriminador_tape.gradient(discriminador_loss, discriminador.trainable_variables)

  gerador_otimizador.apply_gradients(zip(gradientes_gerador, gerador.trainable_variables))
  discriminador_otimizador.apply_gradients(zip(gradientes_discriminador, discriminador.trainable_variables))

In [ ]:
imagens_teste = tf.random.normal([numero_exemplos_para_gerar, ruido_dimensao])
imagens_teste.shape

In [ ]:
def treina_gan(dataset, epochs, imagens_teste):
  for epoca in range(epochs):
    print('Época: ' + str(epoca))
    for imagem_batch in dataset:
      treinamento(imagem_batch)

    imagens_geradas = gerador(imagens_teste, training = False)
    fig = plt.figure(figsize = (10, 10))
    for i in range(imagens_geradas.shape[0]):
      plt.subplot(4, 4, i + 1)
      plt.imshow(imagens_geradas[i, :, :, 0] * 127.5 + 127.5, cmap = 'gray')
      plt.axis('off')
    plt.show()

In [ ]:
treina_gan(x_treinamento, epochs, imagens_teste)